We planned to constrcut a Semantic Search Engine for getting some regulations based on input query using BM-25 and Sentence Formations concept. First step is to load the dataset.
We manually downloaded the dataset from the project document.

In [ ]:
import json

file_path = '/content/ecfr-title-12.jsonl' # loading the jsonl dataset

with open(file_path, 'r') as file:
    count = 0
    for line in file:
        if count >= 5: # counting and taking only five values
            break
        data = json.loads(line)
        print(data) # printing the content
        count += 1

{'text': '', 'meta': {'chapter': ['II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII',

To read the jsonl file using python, First we can upload files via both ways. uploading to session storage or uploading jsonl file after mounting to google drive. We choose first option and uploaded jsonl file into session storage.

Then we read it using python 'loads' method from json module. Since the dataset is very long, we loaded only five entries as sample with following code. Now the step is to install libraries required for the appication.

In [ ]:
!pip install transformers torch flask pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install rank_bm25

Now all the libraries are installed, let us go with writing logics.


In [ ]:
import pandas as pd
import re

def load_data(filepath): # loading the dataset as pandas dataframe
    df = pd.read_json(filepath, lines=True)

    df = df[df['text'].notna()]

    df = df[df['text'] != '']

    return df

def clean_text(text):
    # removing unwanted irrelevant symbols from the dataframe
    cleaned_text = re.sub(r'[^\w\s.,;:!?()-]', '', text)

    return cleaned_text

def preprocess_data(filepath):
    df = load_data(filepath)

    # applying text cleaning to the 'text' column
    df['clean_text'] = df['text'].apply(clean_text)

    return df

# input file path
filepath = '/content/ecfr-title-12.jsonl'
processed_df = preprocess_data(filepath)

# output file path and saving it as a new dataset
output_filepath = '/content/ecfr-title-12_cleaned.jsonl'
processed_df.to_json(output_filepath, orient='records', lines=True)

print(f"DataFrame cleaned and saved !!!") # displaying acknowledgement for users


DataFrame cleaned and saved !!!


But the dataset might have missing values and not applicable values along with some irrelevant text in middle. They will degrade the relevancy of search engine. It is best approach to preprocess the data before taking the dataset for model construction or engines.

Hence, we performed preprocessing the data and we cleaned the data by removing the missing values and not applicable. We gave the cleaned data as input to the model.


In [9]:
import pandas as pd #importing all required modules and libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# defining tokenizing function
def tokenize(text):
    return text.lower().split()

# loading cleaned dataframe
df = pd.read_json("/content/ecfr-title-12_cleaned.jsonl", lines=True)
df['text'] = df['text'].fillna('')

# loading all the cleaned tokens from the dataframe and converting them into a list,
tokenized_corpus = df['text'].apply(tokenize).tolist()

# initializing the BM25 model for intital retrieval
bm25 = BM25Okapi(tokenized_corpus)

# initializing the sentence transformer. We are using the "all-MiniLM-L6-v2" model developed by the Microsoft Research
model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embeddings for all the elements in the dataframe from the dataset
df['embeddings'] = df['text'].apply(lambda x: model.encode(x) if x else np.zeros(model.get_sentence_embedding_dimension()))

# search method to get scores and filter top 5 results first
def combined_search(query):
    # tokenizing and using BM25 to get scores
    query_tokens = tokenize(query)
    bm25_scores = bm25.get_scores(query_tokens)
    top_5_final_indices = np.argsort(bm25_scores)[::-1][:5]

    # getting embeddings of the top 5 regulations from the dataset
    top_5_embeddings = np.array(df.loc[top_5_final_indices, 'embeddings'].tolist())

    # encoding the query using Sentence Transformers
    query_embedding = model.encode(query)

    # calculating similarity based on cosine similarities with top 5 regulations embeddings
    similarities = cosine_similarity([query_embedding], top_5_embeddings)[0]

    return df.loc[top_5_final_indices, ['clean_text', 'meta']]

# taking query dynamically from user in run time
user_query = input("Enter your query: ")
results = combined_search(user_query)
print("Top results:")
print(results)


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Enter your query: requirements for consumer data protection
Top results:
                                            clean_text  \
399  (a) Authority. The regulation in this part, kn...   
914  (a) Entities required to report. Each issuer t...   
366  (a) Authority and scope. This regulation is is...   
440  (a) Authority. This regulation, known as Regul...   
666  (a) Coverage. A financial institution shall co...   

                                                  meta  
399  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
914  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
366  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
440  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
666  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  


This is the main part of the code and the primary logic lies in this code block. We input any words or query about what information we want and it will give five top relevant regulations based on that query using BM-25 and Sentence Transformers concepts with Semantic Search Engine.

First we have imported all the necessary modules and libraries. Then we are converting the text string to lowercase to nomalize the case and it splits the strings into tokens, list of words.

Then we have set up BM-25 model and Sentence Transformer "all-MiniLM-L6-v2" for creating embeddings. Then we created the embeddings for the content of the dataframe taken from the dataset.

After creating the embeddings for the content in the data frames form the dataset, we have taken query from user at run time and converted it into the embeddings.

We compared the both embeddings and calculated the similarity between the embeddings using cosine similarity function and got top five relevant regulations as search results for the input query given.


In [16]:
import pandas as pd #importing all required modules and libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# defining tokenizing function
def tokenize(text):
    return text.lower().split()

# loading cleaned dataframe
df = pd.read_json("/content/ecfr-title-12_cleaned.jsonl", lines=True)
df['text'] = df['text'].fillna('')

# loading all the cleaned tokens from the dataframe and converting them into a list,
tokenized_corpus = df['text'].apply(tokenize).tolist()

# initializing the BM25 model for intital retrieval
bm25 = BM25Okapi(tokenized_corpus)

# initializing the sentence transformer. We are using the "all-MiniLM-L6-v2" model developed by the Microsoft Research
model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embeddings for all the elements in the dataframe from the dataset
df['embeddings'] = df['text'].apply(lambda x: model.encode(x) if x else np.zeros(model.get_sentence_embedding_dimension()))

# search method to get scores and filter top 5 results first
def combined_search(query):
    # tokenizing and using BM25 to get scores
    query_tokens = tokenize(query)
    bm25_scores = bm25.get_scores(query_tokens)
    top_5_final_indices = np.argsort(bm25_scores)[::-1][:5]

    # getting embeddings of the top 5 regulations from the dataset
    top_5_embeddings = np.array(df.loc[top_5_final_indices, 'embeddings'].tolist())

    # encoding the query using Sentence Transformers
    query_embedding = model.encode(query)

    # calculating similarity based on cosine similarities with top 5 regulations embeddings
    similarities = cosine_similarity([query_embedding], top_5_embeddings)[0]

    return df.loc[top_5_final_indices, ['clean_text', 'meta']]

# taking query dynamically from user in run time
user_query = input("Enter your query: ")
results = combined_search(user_query)
print("Top results:")
print(results)

Enter your query: loan
Top results:
                                             clean_text  \
4236  The following transactions require the Boards ...   
2638  (a) The Board recently was asked whether a loa...   
1803  (a) Maximum loan value of margin stock. The ma...   
2979  (a) What happens when a qualified lender sells...   
4537  Under the policies of its board, each institut...   

                                                   meta  
4236  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
2638  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
1803  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
2979  {'chapter': ['VI'], 'chapter_title': ['CHAPTER...  
4537  {'chapter': ['VI'], 'chapter_title': ['CHAPTER...  


In [11]:
import pandas as pd #importing all required modules and libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# defining tokenizing function
def tokenize(text):
    return text.lower().split()

# loading cleaned dataframe
df = pd.read_json("/content/ecfr-title-12_cleaned.jsonl", lines=True)
df['text'] = df['text'].fillna('')

# loading all the cleaned tokens from the dataframe and converting them into a list,
tokenized_corpus = df['text'].apply(tokenize).tolist()

# initializing the BM25 model for intital retrieval
bm25 = BM25Okapi(tokenized_corpus)

# initializing the sentence transformer. We are using the "all-MiniLM-L6-v2" model developed by the Microsoft Research
model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embeddings for all the elements in the dataframe from the dataset
df['embeddings'] = df['text'].apply(lambda x: model.encode(x) if x else np.zeros(model.get_sentence_embedding_dimension()))

# search method to get scores and filter top 5 results first
def combined_search(query):
    # tokenizing and using BM25 to get scores
    query_tokens = tokenize(query)
    bm25_scores = bm25.get_scores(query_tokens)
    top_5_final_indices = np.argsort(bm25_scores)[::-1][:5]

    # getting embeddings of the top 5 regulations from the dataset
    top_5_embeddings = np.array(df.loc[top_5_final_indices, 'embeddings'].tolist())

    # encoding the query using Sentence Transformers
    query_embedding = model.encode(query)

    # calculating similarity based on cosine similarities with top 5 regulations embeddings
    similarities = cosine_similarity([query_embedding], top_5_embeddings)[0]

    return df.loc[top_5_final_indices, ['clean_text', 'meta']]

# taking query dynamically from user in run time
user_query = input("Enter your query: ")
results = combined_search(user_query)
print("Top results:")
print(results)

Enter your query: The purpose of this part is to implement a prohibition against an Enterprise redirecting the cost of any allocation to the Housing Trust Fund or the Capital Magnet Fund to originators of mortgages purchased or securitized by an Enterprise.
Top results:
                                             clean_text  \
4295  The purpose of this part is to implement a pro...   
1623  (a) In general. No Enterprise shall re-direct ...   
2152  (a) Purpose. The purpose of this part is to re...   
1418  (a) General. Subject to FHFAs determination of...   
423   (a) Authority. This part is issued by the Offi...   

                                                   meta  
4295  {'chapter': ['XII'], 'chapter_title': ['CHAPTE...  
1623  {'chapter': ['XII'], 'chapter_title': ['CHAPTE...  
2152  {'chapter': ['XII'], 'chapter_title': ['CHAPTE...  
1418  {'chapter': ['XII'], 'chapter_title': ['CHAPTE...  
423   {'chapter': ['XVII'], 'chapter_title': ['CHAPT...  


In [12]:
import pandas as pd #importing all required modules and libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# defining tokenizing function
def tokenize(text):
    return text.lower().split()

# loading cleaned dataframe
df = pd.read_json("/content/ecfr-title-12_cleaned.jsonl", lines=True)
df['text'] = df['text'].fillna('')

# loading all the cleaned tokens from the dataframe and converting them into a list,
tokenized_corpus = df['text'].apply(tokenize).tolist()

# initializing the BM25 model for intital retrieval
bm25 = BM25Okapi(tokenized_corpus)

# initializing the sentence transformer. We are using the "all-MiniLM-L6-v2" model developed by the Microsoft Research
model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embeddings for all the elements in the dataframe from the dataset
df['embeddings'] = df['text'].apply(lambda x: model.encode(x) if x else np.zeros(model.get_sentence_embedding_dimension()))

# search method to get scores and filter top 5 results first
def combined_search(query):
    # tokenizing and using BM25 to get scores
    query_tokens = tokenize(query)
    bm25_scores = bm25.get_scores(query_tokens)
    top_5_final_indices = np.argsort(bm25_scores)[::-1][:5]

    # getting embeddings of the top 5 regulations from the dataset
    top_5_embeddings = np.array(df.loc[top_5_final_indices, 'embeddings'].tolist())

    # encoding the query using Sentence Transformers
    query_embedding = model.encode(query)

    # calculating similarity based on cosine similarities with top 5 regulations embeddings
    similarities = cosine_similarity([query_embedding], top_5_embeddings)[0]

    return df.loc[top_5_final_indices, ['clean_text', 'meta']]

# taking query dynamically from user in run time
user_query = input("Enter your query: ")
results = combined_search(user_query)
print("Top results:")
print(results)

Enter your query: It is the position of the Board of Governors of the Federal Reserve System that, for purposes of provisions of law administered by the Board, a transaction in Federal funds involves a loan on the part of the selling bank and a borrowing on the part of the purchasing bank
Top results:
                                             clean_text  \
1733  (a) It is the position of the Board of Governo...   
3746  For the purposes of this part:, (a) Board of D...   
778   (a) Delegation of authority. Designated Board ...   
3748  For the purposes of this part:, Affiliated dep...   
2625  (a) The Board of Governors has clarified and m...   

                                                   meta  
1733  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
3746  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
778   {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
3748  {'chapter': ['II'], 'chapter_title': ['CHAPTER...  
2625  {'chapter': ['II'], 'chapter_title': ['CHAPTER

In [13]:
import pandas as pd #importing all required modules and libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

# defining tokenizing function
def tokenize(text):
    return text.lower().split()

# loading cleaned dataframe
df = pd.read_json("/content/ecfr-title-12_cleaned.jsonl", lines=True)
df['text'] = df['text'].fillna('')

# loading all the cleaned tokens from the dataframe and converting them into a list,
tokenized_corpus = df['text'].apply(tokenize).tolist()

# initializing the BM25 model for intital retrieval
bm25 = BM25Okapi(tokenized_corpus)

# initializing the sentence transformer. We are using the "all-MiniLM-L6-v2" model developed by the Microsoft Research
model = SentenceTransformer('all-MiniLM-L6-v2')

# generating embeddings for all the elements in the dataframe from the dataset
df['embeddings'] = df['text'].apply(lambda x: model.encode(x) if x else np.zeros(model.get_sentence_embedding_dimension()))

# search method to get scores and filter top 5 results first
def combined_search(query):
    # tokenizing and using BM25 to get scores
    query_tokens = tokenize(query)
    bm25_scores = bm25.get_scores(query_tokens)
    top_5_final_indices = np.argsort(bm25_scores)[::-1][:5]

    # getting embeddings of the top 5 regulations from the dataset
    top_5_embeddings = np.array(df.loc[top_5_final_indices, 'embeddings'].tolist())

    # encoding the query using Sentence Transformers
    query_embedding = model.encode(query)

    # calculating similarity based on cosine similarities with top 5 regulations embeddings
    similarities = cosine_similarity([query_embedding], top_5_embeddings)[0]

    return df.loc[top_5_final_indices, ['clean_text', 'meta']]

# taking query dynamically from user in run time
user_query = input("Enter your query: ")
results = combined_search(user_query)
print("Top results:")
print(results)

Enter your query: The term savings account means any withdrawable account, except a demand account as defined in § 390.290, a tax and loan account, a note account, a United States Treasury general account, or a United States Treasury time deposit-open account.
Top results:
                                             clean_text  \
4397  The term savings account means any withdrawabl...   
4371  The term account means any savings account, de...   
4401  The term tax and loan account means an account...   
4369  The term United States Treasury General Accoun...   
4370  The term United States Treasury Time Deposit O...   

                                                   meta  
4397  {'chapter': ['III'], 'chapter_title': ['CHAPTE...  
4371  {'chapter': ['III'], 'chapter_title': ['CHAPTE...  
4401  {'chapter': ['III'], 'chapter_title': ['CHAPTE...  
4369  {'chapter': ['III'], 'chapter_title': ['CHAPTE...  
4370  {'chapter': ['III'], 'chapter_title': ['CHAPTE...  
